In [0]:
import xgboost as xgb
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
#from sklearn.learning_curve import validation_curve
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix 
import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

from xgboost.sklearn import XGBClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection  import StratifiedKFold
from scipy.stats import randint, uniform

seed = 342
np.random.seed(seed)

In [2]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#train_path = os.path.join('..','data','train_data_v3_processed.csv')
train_path = os.path.join('drive','My Drive','quora','train_data_v3_processed.csv')
train_data = pd.read_csv(train_path)

In [0]:
test_path = os.path.join('drive','My Drive','quora','test_data_v3_processed.csv')
test_data = pd.read_csv(test_path)

In [0]:
df = train_data#[:10000]

In [5]:
df.head()

,id,question1,question2,is_duplicate,question1_lemma,question1_tag,question2_lemma,question2_tag,fuzz_ratio,fuzz_partial_ratio,...,wc_diff_unique,wc_ratio_unique,wc_diff_unq_stop,wc_ratio_unique_stop,same_start,char_diff,char_diff_unq_stop,total_unique_words,total_unq_words_stop,char_ratio
0,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,what be the step by step guide to invest in sh...,WP VBZ DT NN IN NN NN TO VB IN NN NN IN NN .,what be the step by step guide to invest in sh...,WP VBZ DT NN IN NN NN TO VB IN NN NN .,93,98,...,1,0.916667,1,0.833333,1,7,5,13,7,0.867925
1,1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,what be the story of kohinoor ( koh - i - noor...,WP VBZ DT NN IN NNP -LRB- NNP HYPH NNP HYPH NN...,what would happen if the indian government ste...,WP MD VB IN DT JJ NN VBD DT NNP -LRB- NNP HYPH...,68,78,...,4,1.500000,5,2.250000,1,32,31,16,11,1.727273
2,2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,how can -PRON- increase the speed of -PRON- in...,WRB MD PRP VB DT NN IN PRP$ NN NN IN VBG DT NNP .,how can internet speed be increase by hack thr...,WRB MD VB NN VB VBN IN VBG IN NN .,35,44,...,4,0.714286,1,0.833333,1,10,7,20,9,0.833333
3,3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,why be -PRON- mentally very lonely ? how can -...,WRB VBP PRP RB RB JJ . WRB MD PRP VB PRP .,find the remainder when [ math]23^{24}[/math ]...,VB DT NN WRB -LRB- NN -RRB- VBZ VBN IN CD .,17,22,...,1,0.900000,1,1.250000,0,17,23,19,9,1.425000
4,4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"which one dissolve in water quikly sugar , sal...","WDT CD NN IN NN RB NN , NN , NN CC NN FW NN .",which fish would survive in salt water ?,WDT NN MD VB IN NN NN .,37,55,...,6,0.538462,5,0.500000,1,31,28,18,15,0.515625


In [0]:
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60.
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

In [0]:
x_columns = ['fuzz_ratio',
       'fuzz_partial_ratio', 'fuzz_partial_token_set_ratio',
       'fuzz_partial_token_sort_ratio', 'fuzz_token_set_ratio',
       'fuzz_token_sort_ratio', 'len_q1', 'len_q2', 'diff_len', 'len_char_q1',
       'len_char_q2', 'len_word_q1', 'len_word_q2', 'common_words',
       'avg_word_q1', 'avg_word_q2', 'num_stop_words_q1', 'num_stop_words_q2',
       'numerics_q1', 'numerics_q2', 'uppercase_q1', 'uppercase_q2',
       'softcossim', 'mistakes_q1', 'mistakes_q2', 'spacy_sim',
       'q1_q2_intersect', 'q1_freq', 'q2_freq', 'word_match', 'tfidf_wm',
       'tfidf_wm_stops', 'jaccard', 'wc_diff', 'wc_ratio', 'wc_diff_unique',
       'wc_ratio_unique', 'wc_diff_unq_stop', 'wc_ratio_unique_stop',
       'same_start', 'char_diff', 'char_diff_unq_stop', 'total_unique_words',
       'total_unq_words_stop', 'char_ratio']

y_columns = ['is_duplicate']

data_y = df[y_columns].values
data_x = df[x_columns].values
train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size=0.20, random_state=42)
#train = train.T
#test = test.T

In [0]:
train_global_df = pd.DataFrame(np.concatenate((train_x, train_y), axis=1))
train_global_df.columns = x_columns + y_columns

data_y = train_global_df[y_columns].values
data_x = train_global_df[x_columns].values

train_x, validation_x, train_y, validation_y = train_test_split(data_x, data_y, test_size=0.20, random_state=42)

train_y = train_y.astype('int')
validation_y = validation_y.astype('int')

In [0]:
cv = StratifiedKFold(data_y.ravel(), n_folds=10, shuffle=True, random_state=seed)

In [0]:
# A parameter grid for XGBoost
params_grid = {
        'max_depth': [5, 10, 15, 20],
        'n_estimators': [5,10,25,50],
        'learning_rate': np.linspace(1e-16,1,3)
        }

In [0]:
params_fixed = {
    'objective': 'binary:logistic',
    'silent': 1
}

In [0]:
bst_grid = GridSearchCV(
    estimator=XGBClassifier(**params_fixed, seed=seed),
    param_grid=params_grid,
    cv=cv,
    scoring='accuracy'
)

In [27]:
bst_grid.fit(data_x, data_y)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an err

GridSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[0. 0. ... 0. 1.], n_folds=10, shuffle=True, random_state=342),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=342, silent=1,
       subsample=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [5, 10, 15, 20], 'n_estimators': [5, 10, 25, 50], 'learning_rate': array([1.e-16, 5.e-01, 1.e+00])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [28]:
print("Best accuracy obtained: {0}".format(bst_grid.best_score_))
print("Parameters:")
for key, value in bst_grid.best_params_.items():
  print("\t{}: {}".format(key, value))

Best accuracy obtained: 0.8848248816349285
Parameters:
	learning_rate: 0.5
	max_depth: 5
	n_estimators: 50


In [0]:
boost = XGBClassifier(max_depth=5, learning_rate=0.5, n_estimators=50, seed=seed)

In [0]:
x_columns = ['fuzz_ratio',
       'fuzz_partial_ratio', 'fuzz_partial_token_set_ratio',
       'fuzz_partial_token_sort_ratio', 'fuzz_token_set_ratio',
       'fuzz_token_sort_ratio', 'len_q1', 'len_q2', 'diff_len', 'len_char_q1',
       'len_char_q2', 'len_word_q1', 'len_word_q2', 'common_words',
       'avg_word_q1', 'avg_word_q2', 'num_stop_words_q1', 'num_stop_words_q2',
       'numerics_q1', 'numerics_q2', 'uppercase_q1', 'uppercase_q2',
       'softcossim', 'mistakes_q1', 'mistakes_q2', 'spacy_sim',
       'q1_q2_intersect', 'q1_freq', 'q2_freq', 'word_match', 'tfidf_wm',
       'tfidf_wm_stops', 'jaccard', 'wc_diff', 'wc_ratio', 'wc_diff_unique',
       'wc_ratio_unique', 'wc_diff_unq_stop', 'wc_ratio_unique_stop',
       'same_start', 'char_diff', 'char_diff_unq_stop', 'total_unique_words',
       'total_unq_words_stop', 'char_ratio']

y_columns = ['is_duplicate']



train_x = train_data[x_columns].values
train_y = train_data[y_columns].values

test_x = test_data[x_columns].values

In [14]:
boost.fit(train_x, train_y)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.5, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=50,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=342,
       silent=True, subsample=1)

In [0]:
predicted = boost.predict(test_x)

In [17]:
predicted.astype('int')

81126

81126

In [21]:
create_csvs(predicted.astype('int'), test_data.test_id.values)

saved in:  /content/./submission_0823AM-December-12-2018.csv


In [0]:
import csv
import datetime
from os.path import join,abspath,curdir
import numpy as np
import pandas as pd

def create_csvs(predicted, test_ids):
    EXPECTED_ROWS = 81126 
    tests_ids_len = len(test_ids)
    assert(tests_ids_len == EXPECTED_ROWS)
    assert(len(predicted)==tests_ids_len)
    
    FOLDER = join('.')
    CURRENT_PATH = abspath(curdir)
    
    merged = {'test_id': test_ids}
    merged['is_duplicate'] = predicted

    FILENAME = 'submission_' + datetime.datetime.now().strftime("%I%M%p-%B-%d-%Y") + '.csv'
    df = pd.DataFrame.from_dict(merged)

    df.set_index('test_id', inplace=True)

    FULL_PATH = join(CURRENT_PATH, FOLDER, FILENAME)

    df.to_csv(path_or_buf=FULL_PATH, sep=',')

    print('saved in: ', FULL_PATH)

# **Randomized Grid-Search**

In [1]:
params_dist_grid = {
    'max_depth': [5, 10, 15, 20],
    'gamma': [0, 0.5, 1],
    'n_estimators': randint(1, 1001),
    'learning_rate': uniform(),
    'subsample': uniform(),
    'colsample_bytree': uniform()
}

NameError: ignored

In [2]:
rs_grid = RandomizedSearchCV(
    estimator=XGBClassifier(**params_fixed, seed=seed),
    param_distributions=params_dist_grid,
    n_iter=10,
    cv=cv,
    scoring='accuracy',
    random_state=seed
)

NameError: ignored

In [0]:
rs_grid.fit(data_x, data_y)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an err

In [0]:
rs_grid.best_estimator_

In [0]:
rs_grid.best_params_

In [0]:
rs_grid.best_score_